In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import utils




/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [39]:
# setting
DATE = '812_1'
LOOP = 2
ESR = 40

#seed = np.random.randint(99999)
seed = 71

np.random.seed(seed)

valid_size = 0.05


# XGB param
nround = 10000
nround = 1

param = {'max_depth':10, 
         'eta':0.02,
         'colsample_bytree':0.4,
         'subsample':0.75,
         'silent':1,
         'nthread':27,
         'eval_metric':'logloss',
         'objective':'binary:logistic',
         'tree_method':'auto'
         }

print("""#==== print param ======""")
print('DATE:', DATE)
print('seed:', seed)



#==== print param ======
DATE: 812_1
seed: 71


In [ ]:
#==============================================================================
# prepare
#==============================================================================
train = pd.concat([utils.load_pred_feature('trainW-0', keep_all = True),
                   utils.load_pred_feature('trainW-1',keep_all = True),
                   utils.load_pred_feature('trainW-2',keep_all = True)
                   ], ignore_index=True)


In [ ]:
train.info()

In [ ]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(train)
train.info()

In [ ]:
y_train = train['is_churn']
X_train = train.drop('is_churn', axis=1)
del train
gc.collect()



In [6]:
# drop obj--> that's msno
col = X_train.dtypes[X_train.dtypes=='object'].index.tolist()
col.remove('msno')
print('drop',col)
X_train.drop(col, axis=1, inplace=True)

X_train.fillna(-1, inplace=True)

drop []


In [19]:
'msno' in X_train.columns

True

# train/val splitting

In [20]:
train_user = X_train[['msno']].drop_duplicates()

In [21]:
train_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 996693 entries, 0 to 2672075
Data columns (total 1 columns):
msno    996693 non-null object
dtypes: object(1)
memory usage: 15.2+ MB


In [22]:
train_user[train_user.msno.duplicated()]

,msno


In [11]:
np.random.choice([0,1], size=len(train_user), p=[1-valid_size, valid_size]) 
# p = [0.95,0.05]: the prob that randomly generate 1 is valid_size, 1 m

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
train_user['is_valid'] = np.random.choice([0,1], size=len(train_user), 
                                          p=[1-valid_size, valid_size])
# is_valid: 1 if the user is validating user else 0
valid_n = train_user['is_valid'].sum()
build_n = (train_user.shape[0] - valid_n)

print('build user:{}, valid user:{}'.format(build_n, valid_n))
valid_user = train_user[train_user['is_valid']==1].msno

build user:946438, valid user:50255


In [13]:
is_valid = X_train.msno.isin(valid_user)

In [14]:
is_valid

0          False
1          False
2          False
3          False
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22          True
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
2672047    False
2672048    False
2672049    False
2672050    False
2672051    False
2672052    False
2672053    False
2672054    False
2672055     True
2672056    False
2672057     True
2672058    False
2672059    False
2672060    False
2672061    False
2672062    False
2672063    False
2672064    False
2672065    False
2672066    False
2672067    False
2672068     True
2672069    False
2672070    False
2672071    False
2672072    False
2672073    False
2672074    Fal

In [15]:
    dbuild = xgb.DMatrix(X_train[~is_valid].drop('msno', axis=1), y_train[~is_valid])
    dvalid = xgb.DMatrix(X_train[is_valid].drop('msno', axis=1), label = y_train[is_valid])
    watchlist = [(dbuild, 'build'),(dvalid, 'valid')]
 

In [16]:
watchlist

[(<xgboost.core.DMatrix at 0x112326da0>, 'build'),
 (<xgboost.core.DMatrix at 0x112326240>, 'valid')]

In [24]:
def split_build_valid():
    # train/val split by user
    train_user['is_valid'] = np.random.choice([0,1], size=len(train_user), 
                                              p=[1-valid_size, valid_size])
    # is_valid: 1 if the user is validating user else 0
    valid_n = train_user['is_valid'].sum()
    build_n = (train_user.shape[0] - valid_n)
    
    print('build user:{}, valid user:{}'.format(build_n, valid_n))
    valid_user = train_user[train_user['is_valid']==1].msno
    is_valid = X_train.msno.isin(valid_user)
    
    dbuild = xgb.DMatrix(X_train[~is_valid].drop('msno', axis=1), y_train[~is_valid])
    dvalid = xgb.DMatrix(X_train[is_valid].drop('msno', axis=1), label = y_train[is_valid])
    watchlist = [(dbuild, 'build'),(dvalid, 'valid')]
    
    print('FINAL SHAPE')
    print('dbuild.shape:{}  dvalid.shape:{}\n'.format((dbuild.num_row(), dbuild.num_col()),
                                                      (dvalid.num_row(), dvalid.num_col())))

    return dbuild, dvalid, watchlist

In [33]:
import os
def mkdir_p(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [40]:
DATE = '1212_1'
mkdir_p('../../output/model/{}/'.format(DATE))
mkdir_p('../../output/sub/{}/'.format(DATE))
# hold out
models = [] # for the following prediction
for i in range(LOOP):
    print('LOOP',i)
    dbuild, dvalid, watchlist = split_build_valid()
    
    if i==0:
        col_train = dbuild.feature_names 
        # col_train is built for testing cause u do not use all feature. 
        # we use feature with top importance
        
    model = xgb.train(param, dbuild, nround, watchlist,
                      early_stopping_rounds=ESR, verbose_eval=5)
    models.append(model)
    model.save_model('../../output/model/{}/xgb_churn_{}.model'.format(DATE, i))
    # VALID
    valid_yhat = model.predict(dvalid) # y_hat is result of prediction
    print('Valid Mean:', np.mean(valid_yhat))
    del dbuild, dvalid, watchlist
    gc.collect()
 

LOOP 0
build user:947178, valid user:49515
FINAL SHAPE
dbuild.shape:(2539234, 290)  dvalid.shape:(132843, 290)

[0]	build-logloss:0.676224	valid-logloss:0.676254
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 40 rounds.
Valid Mean: 0.502087


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x120a0d4a8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


LOOP 1
build user:946773, valid user:49920
FINAL SHAPE
dbuild.shape:(2538058, 290)  dvalid.shape:(134019, 290)

[0]	build-logloss:0.676219	valid-logloss:0.676193
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 40 rounds.
Valid Mean: 0.502077


In [42]:
#==============================================================================
print('test')
#==============================================================================
test = utils.load_pred_feature('test', keep_all = True).fillna(-1)

sub_test = test[['msno']]

dtest  = xgb.DMatrix(test[col_train])
sub_test['yhat'] = 0



  0%|          | 0/20 [00:00<?, ?it/s]

test



100%|██████████| 20/20 [00:44<00:00,  2.22s/it]


test.shape:(907471, 293)



In [48]:
for model in models:
    sub_test['yhat'] += model.predict(dtest)
sub_test['yhat'] /= LOOP
print('Test Mean:', sub_test['yhat'].mean())

# sub_test.to_pickle('../output/sub/{}/sub_test.p'.format(DATE))


Test Mean: 0.99008106791


In [47]:
sub_test

,msno,yhat
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0.980669
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,1.000273
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0.981195
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0.980669
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,1.011035
5,nu1jZ/flvSaXRg0U9Es+xmo2KlAXq/q+mhcWFStwm9w=,1.004315
6,biDTtgK83fEWXJDaxzD1eh22dion/h3odeIbS7qJMmY=,1.000093
7,k/8uwi/iM9LZmRAIWXLqpZY6ENomXAscwsQsh6PxcTw=,1.002191
8,18rQ/746SjA6nBx325UsyhfsDhu4tK01FXFxHWZjw20=,0.980669
9,2V13OCoWx6vqKr/ZzNmKFrmnC2FtR4SWMz5C5Hi02PY=,1.000093
